In [1]:
import requests

In [2]:
res = requests.get("https://python.langchain.com/en/latest/")
res

<Response [200]>

In [3]:
from bs4 import BeautifulSoup
import urllib.parse
import html
import re

In [4]:
domain = "https://python.langchain.com/"
domain_full = domain+"en/latest/"

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')

# Find all links to local pages on the website

local_links = []
for link in soup.find_all('a',href=True):
    href=link['href']
    if href.startswith(domain) or href.startswith('./') \
        or href.startswith('/') or href.startswith('modules') \
        or href.startswith('user_cases'):
        local_links.append(urllib.parse.urljoin(domain_full,href))

# Find the main content using CSS selectors
main_content = soup.select('body main')[0]

# Extract the HTML code of the main content
main_content_html = str(main_content)

# Extract the plaintext of the main content
main_content_text = main_content.get_text()

# Remove all HTML tags
main_content_text = re.sub(r'<[^>]+>','',main_content_text)

# Remove extract white space
main_content_text = ' '.join(main_content_text.split())

# Replace HTML entities with their corresponding characters
main_content_text = html.unescape(main_content_text)

print(main_content_text)

.rst .pdf Welcome to LangChain Contents Getting Started Modules Use Cases Reference Docs LangChain Ecosystem Additional Resources Welcome to LangChain# LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an API, but will also: Be data-aware: connect a language model to other sources of data Be agentic: allow a language model to interact with its environment The LangChain framework is designed with the above principles in mind. This is the Python specific portion of the documentation. For a purely conceptual guide to LangChain, see here. For the JavaScript documentation, see here. Getting Started# Checkout the below guide for a walkthrough of how to get started using LangChain to create an Language Model application. Getting Started Documentation Modules# There are several main modules that LangChain provides support for. For each module we provid

In [6]:
def scrape(url: str):
    res = requests.get(url)
    if res.status_code != 200:
        print(f"{res.status_code} for '{url}'")
        return None
    soup = BeautifulSoup(res.text, 'html.parser')

    # Find all links to local pages on the website
    local_links = []
    for link in soup.find_all('a',href=True):
        href=link['href']
        if href.startswith(domain) or href.startswith('./') \
            or href.startswith('/') or href.startswith('modules') \
            or href.startswith('user_cases'):
            local_links.append(urllib.parse.urljoin(domain_full,href))

    # Find the main content using CSS selectors
    main_content = soup.select('body main')[0]

    # Extract the HTML code of the main content
    main_content_html = str(main_content)

    # Extract the plaintext of the main content
    main_content_text = main_content.get_text()

    # Remove all HTML tags
    main_content_text = re.sub(r'<[^>]+>','',main_content_text)

    # Remove extract white space
    main_content_text = ' '.join(main_content_text.split())

    # Replace HTML entities with their corresponding characters
    main_content_text = html.unescape(main_content_text)

    # Return as JSON
    return {
        "url":url,
        "text":main_content_text
    }, local_links

In [7]:
links = ["https://python.langchain.com/en/latest/"]
scraped = set()
data = []

while True:
    if len(links) == 0:
        print("Complete")
        break
    url = links[0]
    print(url)
    res = scrape(url)
    scraped.add(url)
    if res is not None:
        page_content, local_links = res
        data.append(page_content)
        # add new links to links list
        links.extend(local_links)
        # remove duplicates
        links = list(set(links))
    # remove links 
    links = [link for link in links if link not in scraped]

https://python.langchain.com/en/latest/
https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/evernote.html
https://python.langchain.com/en/latest/modules/chains/examples/constitutional_chain.html
https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html
https://python.langchain.com/en/latest/use_cases/summarization.html
https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/copypaste.html
https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html
https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/latex.html
https://python.langchain.com/en/latest/modules/agents/tools.html
https://python.langchain.com/en/latest/modules/models/llms/examples/token_usage_tracking.html
https://python.langchain.com/en/latest/modules/prompts/example_selectors/examples/similarity.html
https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/gitbook.html
https://python.l

In [8]:
import tiktoken

In [9]:
tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 20,
    length_function = tiktoken_len,
    separators=["\n\n", "\n", " ",""]
)

Process the data into more chunks using this approach.

In [11]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record ['url']
    } for i in range(len(texts))])

/Users/taj/development/python/llmchain_docs/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 272/272 [00:01<00:00, 159.88it/s]


Our chunks are ready so now we move onto embedding and indexing everything

# Initialize Embedding Model

We use text-embedding-ada-002 as the embedding model. We can embed text like so.

In [12]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
# environment variables

True

In [13]:
# Initialize openai API key
openai.api_key = os.getenv("OPENAI_API_KEY")

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In the response res we will find a JSON-like object containing our new embeddings within the 'data' field.

In [14]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Inside 'data' we will find two records, one for each of the two sentences we just embedded. Each vector embedding contains 1536 dimensions - we may just switch to another method of embedding from the other video

In [15]:
len(res['data'])

2

In [16]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

 We will apply this same embedding logic to the langchain docs dataset we've just scraped. But before doing so we must create a place to store the embeddings.

# Initializing the Index

Now we need to place to store these embeddings and enable a effecient vector search through them all. To do that we use Pinecone, we can get a free API key and enter it below where we will initialize our connection to Pinecone and create a new index.

In [17]:
import pinecone

In [27]:
index_name = 'gpt-4-langchain-docs'

# Initialize connection to pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment="us-central1-gcp"
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )

# Connect to index
index = pinecone.Index(index_name)

# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

 We can see the index is currenlty empty with a total_vector_count of 0. We can begin populating it with OpenAI text-embedding-ada-002 built embeddings like so

In [28]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100 # how many embedding we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoice RateLimitError)
    try:
        res = openai.Embedding.create(inpute=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url'] 
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)
    

100%|██████████| 13/13 [01:35<00:00,  7.32s/it]


Now we've added all of our langchain docs to the index. With that we can move on to retrieve and then answer generation using GPT-4

# Retrieval

To search through our documents we first need to create a query vector xq. Using xq we retrieve the most relevant chunks from the LancChain docs, like so:

In [36]:
query = "how do I use LLMChain in Langchain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

res

{'matches': [{'id': '8c4edbb5-2cf3-43f8-8376-d10590f0a58f',
              'metadata': {'chunk': 0.0,
                           'text': '.rst .pdf Chains Chains# Note Conceptual '
                                   'Guide Using an LLM in isolation is fine '
                                   'for some simple applications, but many '
                                   'more complex ones require chaining LLMs - '
                                   'either with each other or with other '
                                   'experts. LangChain provides a standard '
                                   'interface for Chains, as well as some '
                                   'common implementations of chains for ease '
                                   'of use. The following sections of '
                                   'documentation are provided: Getting '
                                   'Started: A getting started guide for '
                                   'chains, to get you u

With retrieval complete, we move on to fedding these into GPT-4 to produce answers.

# Retrieval Augmented Generation

GPT-4 is currently accessed via the ChatCompletions endpoint of OpenAI. To add the information we retrieved into the model, we need to pass it into our user prompts alongside our original query. We can do that like so:

In [37]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [32]:
print(augmented_query)

.rst .pdf Chains Chains# Note Conceptual Guide Using an LLM in isolation is fine for some simple applications, but many more complex ones require chaining LLMs - either with each other or with other experts. LangChain provides a standard interface for Chains, as well as some common implementations of chains for ease of use. The following sections of documentation are provided: Getting Started: A getting started guide for chains, to get you up and running quickly. How-To Guides: A collection of how-to guides. These highlight how to use various types of chains. Reference: API reference documentation for all Chain classes. previous Redis Chat Message History next Getting Started By Harrison Chase © Copyright 2023, Harrison Chase. Last updated on Apr 03, 2023.

---

and more. LangChain provides a large collection of common utils to use in your application.\nChains: Chains go beyond just a single LLM call, and are sequences of calls (whether to an LLM or a different utility). LangChain prov

Now we ask the question:

In [38]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers user questions based on the information provided by the user above each question. If the information can not be found in the information provided by the user you truthfully say "I don't know"."""

In [39]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

To display this response nicely, we will display it in markdown

In [40]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

To use the LLMChain in LangChain, follow these steps:

1. Import the necessary modules and classes:

```python
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
```

2. Create and configure an LLM instance (for this example, we use OpenAI):

```python
llm = OpenAI(temperature=0.9)
```

3. Create a prompt template with input variables:

```python
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good description of the product {product}?"
)
```

4. Initialize the LLMChain with the LLM instance and the prompt template:

```python
chain = LLMChain(llm=llm, prompt_template=prompt)
```

5. Pass the input as a dictionary to the `run` method of the LLMChain and retrieve the response:

```python
input_data = {"product": "smartphone"}
response = chain.run(input_data)
print(response)
```

This example demonstrates how to use the LLMChain in LangChain with an OpenAI LLM and a custom prompt template. Adapt the LLM instance or the prompt template to suit your specific use case.

Let's compare this to a non-augmented query...

In [42]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

I don't know.

What if we dropped the "I don't know" part of the primer?